# GCS model in Python
## Solving Gouy-Chapman-Stern (GCS) numerically from scratch

In today's exercise, we will seek to solve the Gouy-Chapman-Stern Boundary Value Problem (BVP) numerically:

$$
\text{GCS (1D):} 
\begin{cases} 
\frac{\text{d}^{2}\phi}{\text{d}x^{2}} = \frac{F c_{b}}{\epsilon_{s}}\sinh(\phi), & X \in ]0, L_{\text{max}}[ \\
\phi(x = L_{\text{min}}) = \phi_{\text{HP}}, & X = L_{\text{min}} \tag{1} \\
\lim_{x \to L_{\text{max}}} \phi(x = L_{\text{max}}) = 0, & X \to L_{\text{max}}
\end{cases}
$$

where F is the Faraday constant, $\approx 9.649\cdot 10^{4} \text{ C}\cdot\text{mol}^{-1}$, $\epsilon_{\text{S}}$ is the relative permittivity (old: dielectric constant) of the solvent with respect to electric permittivity of a vacuum, $\epsilon_{\text{S}}\approx 78.4$ (for pure water), $c_{b}$ is the bulk concentration (Molar, M or millimolar, mM), $\phi$ is the electric potential (Volt, V), $x$ is the distance from the Helmholtz Plane (nm).
<p><p>
We will adopt the Newton-Raphson method and benchmark (compare) using the analytical solution to the problem:

$$
\begin{align}
\phi = 2\, \ln\left(\frac{\sqrt{\exp\left(\frac{F\phi_{\text{HP}}}{RT}\right)} - \tanh\left(-x \sqrt{\frac{2 F^{2}c_{b} }{RT\epsilon_{s}}}\right) }{1 - \sqrt{\exp\left(\frac{F\phi_{\text{HP}}}{RT}\right)}\tanh\left(-x \sqrt{\frac{2 F^{2}c_{b} }{RT\epsilon_{s}}}\right) }\right) \tag{2}
\end{align}
$$

where $R$ is the gas constant, $\approx 8.3145 \text{ J}\cdot \text{mol}^{-1}\cdot\text{K}^{-1}$, and $T$ is the temperature (Kelvin, K).

### Setup parameters

The first step, and probably the easiest, is to define the parameters of the model. This can be done in a simple script. But let's define it using classes. This way, we can also use getters (get the values) and setters (restrictions imposed using physical intuition). Example: We get the temperature. And we know that the temperature, in Kelvin, can only take positive real numbers. If at any point we impose a negative value temeprature, we immediatly raise an error.

In [1]:
# Fundamental constants (these are ALWAYS the same)
NA = 6.02214076e23  # Avogadro's number [1/mol]
R = 8.3145          # Gas constant [J/mol·K]
F = 9.649e4         # Faraday constant [C/mol]

class Params:
    
    # Valid constructor arguments
    valid_keys = ['T', 'epsilon_r', 'concentration_mM', 'phi0', 'pzc', 'N', 'L_min', 'L_max']
    
    """
    Parameters for Gouy-Chapman-Stern simulation.

    Parameters
    ----------
    T : float
        Temperature in Kelvin.
    epsilon_r : float
        Relative permittivity of the medium.
    concentration_mM : float
        Bulk electrolyte concentration in millimolar (mM).
    phi0 : float
        Surface potential at the boundary (dimensionless).
    pzc : float
        Point of zero charge (reference potential).
    N : int
        Number of grid points in the spatial domain.
    L_min : float
        Physical minimum length of the domain in meters.
    L_max : float
        Physical maximum length of the domain in meters.

    Attributes
    ----------
    L_min : float
        Minimum spatial domain length (in meters), set via `set_domain_bounds()`.
    L_max : float
        Maximum spatial domain length (in meters), set via `set_domain_bounds()`.
    c_b_mol : float
        Bulk concentration in mol/cm³, converted from millimolar to molar.
    n_b : float
        Number density in 1/cm³, computed using Avogadro's number.
    """
    def __init__(self, T=298.15, epsilon_r=78.4, concentration_mM=100.0, 
                 phi0=-1.2, pzc=0.0, N=2**6, L_min=0.0, L_max = 5e-9):
        """
        Initialize simulation parameters for the GCS model.

        Sets physical constants, simulation domain bounds, and computes
        derived properties such as molar concentration and particle density.
        """
        self.T = T
        self.epsilon_r = epsilon_r
        self.concentration_mM = concentration_mM
        self.phi0 = phi0
        self.pzc = pzc
        self.N = N

        self.set_domain_bounds(L_min, L_max)
        self.c_b_mol = self.convert_mM_to_M(concentration_mM)
        self.n_b = self.calculate_particle_density(self.c_b_mol)
    
    @property
    def T(self):
        """Return the temperature in Kelvin."""
        return self._T

    @T.setter
    def T(self, value):
        """
        Set the temperature (Kelvin).

        Parameters
        ----------
        value : float
            The temperature in Kelvin. Must be greater than zero.

        Raises
        ------
        ValueError
            If the temperature is less than or equal to zero.
        """
        if value <= 0:
            raise ValueError("Temperature must be positive.")
        self._T = value

    @property
    def epsilon_r(self):
        """Return the relative permittivity of the medium."""
        return self._epsilon_r

    @epsilon_r.setter
    def epsilon_r(self, value):
        """
        Set the relative permittivity.

        Parameters
        ----------
        value : float
            The relative permittivity of the medium. Must be greater than zero.

        Raises
        ------
        ValueError
            If the relative permittivity is less than one.
        """
        if value < 1:
            raise ValueError("Permittivity must be greater than 1.")
        self._epsilon_r = value

    @property
    def concentration_mM(self):
        """Return the bulk concentration in millimolar."""
        return self._concentration_mM

    @concentration_mM.setter
    def concentration_mM(self, value):
        """
        Set the bulk concentration in millimolar.

        Parameters
        ----------
        value : float
            The concentration in millimolar (mM). Must be greater than zero.

        Raises
        ------
        ValueError
            If the concentration is less than or equal to zero.
        """
        if value <= 0:
            raise ValueError("Concentration must be positive.")
        self._concentration_mM = value

    @property
    def N(self):
        """Return the number of grid points in the spatial domain."""
        return self._N

    @N.setter
    def N(self, value):
        """
        Set the number of grid points.

        Parameters
        ----------
        value : int
            The number of grid points. Must be a positive integer.

        Raises
        ------
        ValueError
            If the number of grid points is not a positive integer.
        """
        if not isinstance(value, int) or value <= 0:
            raise ValueError("Number of grid points must be a positive integer.")
        self._N = value
    
        
    @property
    def L_min(self):
        """Return the minimum length of the domain in meters."""
        return self._L_min

    @property
    def L_max(self):
        """Return the maximum length of the domain in meters."""
        return self._L_max
    
    def set_domain_bounds(self, L_min, L_max):
        """
        Set the spatial domain boundaries for the simulation.

        This method assigns the minimum and maximum lengths of the spatial domain
        and checks that the values are physically valid. It ensures that `L_min` is
        strictly less than `L_max` to avoid degenerate or reversed domains.

        Parameters
        ----------
        L_min : float
            The minimum boundary of the spatial domain, in meters.
        L_max : float
            The maximum boundary of the spatial domain, in meters.

        Raises
        ------
        ValueError
            If `L_min` is greater than or equal to `L_max`.
        """
        if L_min >= L_max:
            raise ValueError("L_min must be less than L_max.")
        self._L_min = L_min
        self._L_max = L_max

    def convert_mM_to_M(self, concentration_mM):
        """Converts concentration from mM to M."""
        return concentration_mM * 1e-3

    def calculate_particle_density(self, concentration_molar):
        """Calculates number density of particles (1/cm³)."""
        return concentration_molar * NA

### Parameter Class check
If we do not check if what we are doing works, then we cannot really blame anyone but ourselves, if the code does not work down the line. Let us check if this satistifies our expectations. For this reason, I want you to answer the following questions using the code snippet below:

- What would you do in order to fix the code snippet? 
- How would you change the number of grid points to 256? 
- What would do, if you **only** wanted default values?

In [5]:
# Check if the parameters are allowed

#----- YOUR INPUT -----#
T = -300

epsilon_r = 80

L_minnn = 0

L_max = -5e-9

# ?

#----- YOUR INPUT -----#


# Get all valid keys based on the params class
valid_keys = Params.valid_keys  # Defined inside the class
inputs = {key: globals()[key] for key in valid_keys if key in globals()}

# Employ try and except to show succesfull values
# And tell user if any errors are arised.
try:
    #overwrite 
    params = Params(**inputs)
    
    #compare with default values
    default = Params()
    
    print("Params object created successfully.")
    print("Overwritten parameters:")
    for key in valid_keys:
        val = getattr(params, key)
        if key in inputs:
            print(f"  {key} = {val}  (user-defined)")
        else:
            print(f"  {key} = {val}  (default)")
except ValueError as e:
    print(f"Caught an error: {e}")
except TypeError as e:
    print("TypeError: you misspelled a parameter name")
    print("", e)

Params object created successfully.
Overwrittes parameters:
  T = 300  (user-defined)
  epsilon_r = 80  (user-defined)
  concentration_mM = 100.0  (default)
  phi0 = -1.2  (default)
  pzc = 0.0  (default)
  N = 256  (default)
  L_min = 0  (user-defined)
  L_max = 5e-09  (user-defined)


# From real world to numerical
The Newton<span>&ndash;</span>Raphson method is a method used to solve nonlinear differential equations. But in order to employ it, the differential equation in question, eq. (1), ought to be discretized.

## Discretization
Everything in the physical world is either continous and smooth, discrete or somewhere inbetween. Unfortunately, computers only understadnd the discretized world. Therefore, if we wish to numerically/ computationally solve GCS (or any differential equation for that matter), we must employ a method, which rewrites any $\frac{d^{n} \phi}{\text{d}x^{2} }$ terms in a way that the computer will understand. This is called discretization. For today's exercises, we employ a centered second 


## Newton<span>&ndash;</span>Raphson method

# Dimensionless form

Instead, by employing 

$$
\text{GCS (1D):} \quad
\begin{cases}
    \frac{\text{d}^{2}U}{\text{d}X^{2}} = \sinh(U), & \quad X \in ]0, \infty[ \\
    U(0) = U_{\text{HP}}, & \quad X = 0 \tag{3} \\
    \lim_{x \to \infty} U(X) = 0, & \quad X \to \infty
\end{cases}
$$

Where the following transformations have been imposed:

- $ \phi = \frac{RT}{F} U $
- $ x = \lambda_{\text{D}}\, X $
- $ \lambda_{D} = \sqrt{\frac{RT\epsilon_{S}}{2F^{2}c_{b} }} $



Now, that we have these transformations, we simply need to get the transform the parameters and functions to dimensionless form **before** running our solver. After retrieving the solution, we will transform the dimensionless form back to non-dimensionless form and plot it. But how?
- $U = ?$
- $X = ?$

Now you can refer to [Equation 2](#eq:gcs) in the text. [link text](#abcde)

In [1]:
print("Hello friend")

Hello friend


In [2]:
print("Yoooooooo boyyy")

Yoooooooo boyyy
